# Task 2a: Object Detection with Faster R-CNN

**Student:** Stefania Livori  
**Model:** Faster R-CNN (ResNet50 FPN)

This notebook implements a Faster R-CNN object detector to detect Maltese traffic signs. It covers:
1.  **Dataset Preparation**: Loading images and annotations.
2.  **Model Training**: Fine-tuning a pre-trained Faster R-CNN model.
3.  **Evaluation**: Calculating F1-Scores.
4.  **Inference & Analytics**: visualizing detections and counting signs per image.

In [ ]:
# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
from PIL import Image
import json, os
import matplotlib.pyplot as plt
from stefania_livori_utils import *

# Ensure reproducible results
torch.manual_seed(42)

## 1. Dataset Preparation

I define the `SignsDataset` class to load images and COCO-style annotations. 
**Note:** Images are stored in `data/` and annotations in `json_stefania.json`.

In [ ]:
# Update paths here to match your environment
DATA_DIR = "label-studio/label-studio/media/upload/2" 
ANNOTATION_FILE = "stefania_livori/result.json"

In [ ]:
CLASS_NAME_TO_ID = {
    "Background": 0,
    "Stop": 1,
    "No Entry (One Way)": 2,
    "Pedestrian Crossing": 3,
    "Roundabout Ahead": 4,
    "No Through Road (T-Sign)": 5,
    "Blind-Spot Mirror (Convex)": 6
}

# Reverse mapping for visualization
CLASS_NAMES = {v: k for k, v in CLASS_NAME_TO_ID.items()}

# +1 for background
NUM_CLASSES = len(CLASS_NAME_TO_ID) + 1  

# https://medium.com/@RobuRishabh/understanding-and-implementing-faster-r-cnn-248f7b25ff96
class SignsDataset(Dataset):
    def __init__(self, root, annFile, transforms=None, preload=True):
        self.root = root
        self.transforms = transforms

        # Load the JSON file
        with open(annFile) as f:
            data = json.load(f)

        # Extract images and annotations
        self.images_info = data["images"]
        self.annotations = data["annotations"]

        # Map image_id to annotations for faster access
        self.imgToAnns = {img["id"]: [] for img in self.images_info}
        for ann in self.annotations:
            self.imgToAnns[ann["image_id"]].append(ann)

        self.preload = preload
        # If preload is True, load all images into memory
        if preload:
            # Preload the images into memory for speed
            # This might cause memory usage
            self.loaded_images = []
            for img_info in self.images_info:
                # Handle potential path differences if filename contains folders
                # Assuming images are directly in root or filename matches relative structure
                img_name = os.path.basename(img_info["file_name"])
                img_path = os.path.join(root, img_name)
                
                try:
                    with Image.open(img_path) as img:
                        self.loaded_images.append(img.convert("RGB").copy())
                except FileNotFoundError:
                    # Error if an image is not found
                    raise FileNotFoundError(f"Image not found at {img_path}")

    def __getitem__(self, idx):
        img_info = self.images_info[idx]
        image_id = img_info["id"]

        if self.preload:
            # load the image 
            img = self.loaded_images[idx].copy()
        else:
            img_name = os.path.basename(img_info["file_name"])
            img_path = os.path.join(self.root, img_name)
            img = Image.open(img_path).convert("RGB")

        anns = self.imgToAnns[image_id]
        boxes = []
        labels = []

        for ann in anns:
            x, y, w, h = ann["bbox"]
            # Based on the coco definition
            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"]+1)

        # Guard against images with no annotations if necessary
        # if not boxes:
        #      # Handle empty target
        #      boxes = torch.zeros((0, 4), dtype=torch.float32)
        #      labels = torch.zeros((0,), dtype=torch.int64)
        # else:
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            # Converted to pytorch tensor
            "image_id": torch.tensor([image_id])
        }

        if self.transforms:
            img = self.transforms(img)

        # For debugging: print target labels in torch tensor format
        # print(torch.unique(target["labels"]))

        return img, target

    def __len__(self):
        return len(self.images_info)

### Initialize Dataset and DataLoaders

In [ ]:
# No data augmentation for now
transform = T.Compose([
    T.ToTensor(),
])

dataset = SignsDataset(
    root=DATA_DIR,
    annFile=ANNOTATION_FILE,
    transforms=transform
)

# Split dataset into training and validation sets
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
# Partitions the dataset into training and validation sets once and keeps as it is to have the validation set fixed and unseen
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

# https://github.com/GirinChutia/FasterRCNN-Torchvision-FineTuning/blob/main/train.py
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    # shuffle the dataset
    shuffle=True,
    # Batch collate function to handle variable-size images
    collate_fn=lambda x: tuple(zip(*x))
)

val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    # Batch collate function to handle variable-size images
    collate_fn=lambda x: tuple(zip(*x))
)

## 2. Model Configuration
Loading the Faster R-CNN model pre-trained on COCO and adapting proper number of classes.

In [ ]:
device = get_device()
model = get_faster_rcnn(NUM_CLASSES).to(device)

# Optimizer & Scheduler
# https://github.com/GirinChutia/FasterRCNN-Torchvision-FineTuning/blob/main/train.py
# optimizer logic based on the github link above
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.005,
    # Momentum included to speed up training
    momentum=0.9,
    weight_decay=0.0005
)


# Might need to try different step sizes
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=3, gamma=0.1
)

## 3. Training Loop
Training for 25 epochs and evaluating on validation set after each epoch.

In [ ]:
# number of epochs - needs to be set to 25
num_epochs = 1

# Training Loop
# https://medium.com/@RobuRishabh/understanding-and-implementing-faster-r-cnn-248f7b25ff96 
for epoch in range(num_epochs):
    print(f"\nStarting Epoch {epoch+1}/{num_epochs}")
    avg_loss = train_one_epoch(model, train_loader, optimizer, device)
    scheduler.step()
    
    # Evaluate
    f1 = f1_score_by_iou(model, val_loader, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} | f1-score: {f1:.4f}")

print("Training finished!")

## 4. Analytics & Visualization

Analyze the detections on the validation set. I will count the number of detected signs and visualize the results. I apply inference on the validation set which are unlabelled unseen examples.

In [ ]:
model.eval()

# Analytic - Count detections per image in validation set
# print("GT labels:", target["labels"])
# print("Pred labels:", pred["labels"][:5])
# print("Scores:", pred["scores"][:5])
total_detected_signs = 0
results_summary = []

with torch.no_grad():
    for i, (img, target) in enumerate(val_dataset):
        # Inference
        prediction = model([img.to(device)])[0]
        
        # Evaluate based on the threshold
        keep = prediction["scores"] > 0.3
        final_boxes = prediction["boxes"][keep]
        final_labels = prediction["labels"][keep]
        final_scores = prediction["scores"][keep]
        
        count = len(final_labels)
        total_detected_signs += count
        
        results_summary.append({
            "ImageID": target["image_id"].item(),
            "DetectedSigns": count,
            "Labels": [CLASS_NAMES.get(l.item(), "Unknown") for l in final_labels]
        })

print(f"Total signs detected in validation set: {total_detected_signs}")
print("Results summary:")
for res in results_summary:
    print(res)

### Save the model

In [ ]:
torch.save(model.state_dict(), "./faster_rcnn_stefania_livori.pt")

### Visualisation

In [ ]:
print("\nVisualizing Sample Predictions:")
for i in range(len(val_dataset)):
    img, target = val_dataset[i]
    with torch.no_grad():
        prediction = model([img.to(device)])[0]

    print(f"Image {i+1} Ground Truth: {[CLASS_NAMES[l.item()] for l in target['labels']]} ")
    visualize_predictions(img, prediction, CLASS_NAMES, threshold=0.3)